# QuizKnock_PuzzleKing-vs-Program

元ネタ動画: [【大決戦】コンピュータを使ってでもパズル王に勝ちたい！！](https://www.youtube.com/watch?v=4mh9qsH0Zhs&t=40s)


## 逆ポーランド記法の演算処理

In [1]:
# 有理数の計算をサポートするライブラリ
# 計算結果が分数になった際、除算による計算誤差が発生してしまうため、このライブラリを使用して対応します。
# https://docs.python.org/ja/3/library/fractions.html
from fractions import Fraction

### ベースの実装

In [2]:
def calc_RPN_simple(rpn: str) -> Fraction:
    def ope(a, b, op):
        match op:
            case "+": return a + b
            case "-": return a - b
            case "*": return a * b
            case "/": return a / b

    ops = "+-*/"
    stack = []

    for i, c in enumerate(rpn):
        # 数値(演算子ではない)の場合、スタックに格納
        if c not in ops:
            stack.append(Fraction(int(c), 1))
        # 演算子の場合は、スタックから数値を2つ取り出して演算する
        else:
            var1 = stack.pop()
            var2 = stack.pop()

            # 計算を実施し、結果をスタックに格納
            result = ope(var2, var1, c)
            stack.append(result)

    # 計算結果を返却
    return stack[0]

In [3]:
print(calc_RPN_simple("34+"))
print(calc_RPN_simple("34+12-*"))

7
-7


### 最終的なの実装

In [4]:
def calc_RPN(rpn: str, return_formula: bool=False) -> Fraction|float|str:
    """逆ポーランド記法の演算を実施する

    Args:
        rpn (str): 逆ポーランド記法
        return_formula (bool): Trueの場合は計算結果、Falseの場合は四則演算式の文字列を返却 (Default: False)

    Returns:
        Fraction|float|str: 演算結果(ゼロ割があった場合はfloat("inf")) or 四則演算式の文字列
    """
    def ope(a, b, op):
        match op:
            case "+": return a + b
            case "-": return a - b
            case "*": return a * b
            case "/": return a / b if b != 0 else float("INF")
            case _:
                raise ValueError(f"op(char) is invalid value (value: {op})")

    ops = "+-*/"
    stack = []
    for i, c in enumerate(rpn):
        # 数値(演算子ではない)の場合、スタック
        if c not in ops:
            stack.append(Fraction(int(c), 1))
        # 演算子の場合は、スタックから数値を2つ取り出して演算する
        else:
            # 演算子に対して、数値が不足している場合は例外を出す
            if len(stack) < 2:
                raise ValueError(f"There is an error in reverse polish notation (notation: {rpn}, index: {i}, char: {c})")

            var1 = stack.pop()
            var2 = stack.pop()

            # 式文字列を返却するときの処理
            if return_formula:
                result = f"{var2} {c} {var1}"
                if i < len(rpn) - 1:
                    result = f"({result})"
                stack.append(result)
            # 計算結果を返却するときの処理
            else:
                result = ope(var2, var1, c)
                # 演算結果が無限(ゼロ割)の場合、無限を返却
                if result == float("inf"):
                    return float("inf")
                stack.append(result)

    # 数値に対して、演算子が不足している場合は例外を出す
    if len(stack) > 1:
        raise ValueError("Insufficient number of operators in reverse polish notation")

    # 計算結果 もしくは 数式を返却
    return stack[0] if return_formula else str(stack[0])

### 動作確認

In [5]:
def check(rpn):
    print(f"{calc_RPN(rpn, True)} = {calc_RPN(rpn)}")

In [6]:
check("12+")
check("123+*")
check("321+/")
check("13/23/+")
check("13/23/-")
check("13/32/*")

1 + 2 = 3
1 * (2 + 3) = 5
3 / (2 + 1) = 1
(1 / 3) + (2 / 3) = 1
(1 / 3) - (2 / 3) = -1/3
(1 / 3) * (3 / 2) = 1/2


## 逆ポーランド記法の全列挙

In [7]:
def gen_RPN_all(input_nums: list[int | str]) -> list[str]:
    """入力した数値から逆ポーランド記法を全列挙する

    Args:
        input_nums (list[int | str]): 入力数値リスト(数値は2つ以上)

    Returns:
        list[str]: 全列挙した逆ポーランド記法
    """
    n_nums = len(input_nums)    # 入力数値の個数
    n_ops = n_nums - 1          # 演算子の個数
    n_rpn = n_nums + n_ops      # 逆ポーランド記法の文字数

    used = [False] * n_nums     # 使用したinput_numsを記憶する変数
    rpns = []                   # 生成した逆ポーランド記法を保存する変数

    # DFSにより、あり得る逆ポーランド記法を探索し、rpnsに格納する
    # rpn: 作成中の逆ポーランド記法
    # n_stack: スタックしている数値の個数
    def dfs(rpn="", n_stack=0):
        nonlocal rpns, used

        # 逆ポーランド記法を生成できたタイミングでrpnsに格納
        # (記事内のコードはprintだが、確認の都合上リストに格納する処理としている)
        if len(rpn) >= n_rpn:
            rpns.append(rpn)

        appeared = set()        # 重複防止のために、出現した数値を保存する変数

        for i, num in enumerate(input_nums):
            # 未使用 かつ 未出現の数値の場合、追加可能
            if (not used[i]) and (num not in appeared):
                appeared.add(num)
                used[i] = True
                dfs(rpn + str(num), n_stack + 1)
                used[i] = False

        # スタックが2つ以上ある場合は演算子が追加可能
        if n_stack >= 2:
            for op in "+-*/":
                dfs(rpn + op, n_stack - 1)

    dfs()
    return rpns

### 動作確認

In [8]:
def check(input_nums):
    rpns = gen_RPN_all(input_nums)
    print(f"num: {len(rpns)}, rpns: {rpns}")

In [9]:
check([1, 2])
check([1, 1])
check(["1", "2", "3"])

num: 8, rpns: ['12+', '12-', '12*', '12/', '21+', '21-', '21*', '21/']
num: 4, rpns: ['11+', '11-', '11*', '11/']
num: 192, rpns: ['123++', '123+-', '123+*', '123+/', '123-+', '123--', '123-*', '123-/', '123*+', '123*-', '123**', '123*/', '123/+', '123/-', '123/*', '123//', '12+3+', '12+3-', '12+3*', '12+3/', '12-3+', '12-3-', '12-3*', '12-3/', '12*3+', '12*3-', '12*3*', '12*3/', '12/3+', '12/3-', '12/3*', '12/3/', '132++', '132+-', '132+*', '132+/', '132-+', '132--', '132-*', '132-/', '132*+', '132*-', '132**', '132*/', '132/+', '132/-', '132/*', '132//', '13+2+', '13+2-', '13+2*', '13+2/', '13-2+', '13-2-', '13-2*', '13-2/', '13*2+', '13*2-', '13*2*', '13*2/', '13/2+', '13/2-', '13/2*', '13/2/', '213++', '213+-', '213+*', '213+/', '213-+', '213--', '213-*', '213-/', '213*+', '213*-', '213**', '213*/', '213/+', '213/-', '213/*', '213//', '21+3+', '21+3-', '21+3*', '21+3/', '21-3+', '21-3-', '21-3*', '21-3/', '21*3+', '21*3-', '21*3*', '21*3/', '21/3+', '21/3-', '21/3*', '21/3/', '231+

## QuizKnockパズル用ソルバー

In [10]:
def solver(input_nums: list[int | str], create_num: int) -> str|None:
    """
    複数の数値と四則演算を用いて、指定した数値を作成するソルバー

    Args:
        input_nums (list[int | str]): 入力数値リスト(数値は2つ以上)
        create_num (int): 作成する数値

    Returns:
        str|None: 指定した数値を作成できる四則演算式の文字列 (解がない場合はNone)
    """
    n_nums = len(input_nums)    # 入力数値の個数
    n_ops = n_nums - 1          # 演算子の個数
    n_rpn = n_nums + n_ops      # 逆ポーランド記法の文字数

    used = [False] * n_nums     # 使用したinput_numsを記憶する変数

    # DFSにより、あり得る逆ポーランド記法を探索し、create_numと一致する式の場合は、その式(文字列)を返却する
    # rpn: 作成中の逆ポーランド記法
    # n_stack: スタックしている数値の個数
    def dfs(rpn:str = "", n_stack: int = 0) -> str|None:
        nonlocal used

        # 生成した逆ポーランド記法の計算を実施し、create_numと一致した式(文字列)を返却
        if len(rpn) >= n_rpn:
            if calc_RPN(rpn) == str(create_num):
                return calc_RPN(rpn, return_formula=True)
            else:
                return None

        appeared = set()        # 重複防止のために、出現した数値を保存する変数

        for i, num in enumerate(input_nums):
            # 未使用 かつ 未出現の数値の場合、追加可能
            if (not used[i]) and (num not in appeared):
                appeared.add(num)
                used[i] = True
                formula = dfs(rpn + str(num), n_stack + 1)
                # 該当する式が見つかった場合は式を返却
                if formula:
                    return formula
                used[i] = False

        # スタックが2つ以上ある場合は演算子が追加可能
        if n_stack >= 2:
            for op in "+-*/":
                formula = dfs(rpn + op, n_stack - 1)
                # 該当する式が見つかった場合は式を返却
                if formula:
                    return formula

        # create_numと一致する式が見つからない場合はNoneを返却
        return None

    return dfs()

### 動作確認

In [11]:
def check(input_nums, create_num):
    print(f"{create_num} = {solver(input_nums, create_num)}")

In [12]:
check([1, 2, 3, 4], 15)
check(["1", "2", "3", "4"], "15")

15 = 1 + (2 * (3 + 4))
15 = 1 + (2 * (3 + 4))


## 実際に動画の問題を解いてみる

In [13]:
# 問題 (問題番号、入力となる数値、作成する数値)
QUERY = [
    (1, [1, 2, 3, 4], 15),        (2, [4, 8, 8, 9], 23),        (3, [3, 5, 8, 9], 24),     (4, [1, 3, 5, 7], 19),        (5, [1, 2, 2, 6, 7, 9], 16),
    (6, [1, 2, 3, 3, 3, 9], 13),  (7, [1, 4, 5, 6, 6, 6], 9),   (8, [1, 5, 6, 7], 13),     (9, [1, 2, 3, 6, 6], 24),     (10, [2, 3, 4, 4], 19),
    (11, [2, 4, 7, 9, 9], 14),    (12, [3, 4, 7, 8, 8, 9], 8),  (13, [1, 3, 4, 6], 9),     (14, [2, 3, 9, 9], 14),       (15, [1, 3, 4, 3, 5], 22),
    (16, [1, 4, 6, 6], 8),        (17, [1, 1, 4, 6, 6, 8], 17), (18, [4, 4, 5, 7, 8], 16), (19, [3, 3, 4, 7], 25),       (20, [2, 2, 6, 8, 9], 17),
    (21, [1, 8, 9, 9], 16),       (22, [2, 5, 5, 6, 6, 8], 15), (23, [2, 2, 3, 6, 8], 12), (24, [6, 6, 7, 8, 9], 22),    (25, [1, 6, 8, 8], 18),
    (26, [2, 5, 7, 7, 8], 18),    (27, [3, 3, 4, 8], 19),       (28, [5, 6, 9, 9], 8),     (29, [1, 2, 3, 4, 8, 9], 22), (30, [1, 2, 7, 8], 17),
    (31, [1, 1, 3, 6, 9], 14),    (32, [2, 6, 7, 9], 21),       (33, [1, 4, 6, 7], 23),    (34, [1, 4, 4, 6, 8], 12),    (35, [1, 3, 5, 9], 14),
    (36, [1, 1, 3, 5, 5, 7], 24), (37, [2, 2, 5, 6], 12),       (38, [1, 6, 8, 8], 18),    (39, [1, 3, 4, 6], 13),       (40, [2, 6, 6, 8, 8], 19),
    (41, [3, 4, 4, 9], 11),       (42, [1, 4, 6, 9], 17),       (43, [4, 7, 8, 8], 10),    (44, [1, 6, 8, 9], 18),       (45, [3, 4, 6, 8], 18),
    (46, [1, 2, 4, 8, 9], 13),    (47, [3, 7, 8, 8], 18),       (48, [1, 2, 3, 3, 6], 18), (49, [4, 5, 9, 9, 9], 17),    (50, [5, 6, 7, 9], 19),
    (51, [3, 6, 6, 8], 21),       (52, [1, 2, 7, 7, 8], 17),    (53, [2, 3, 6, 6], 19)
]

In [14]:
%%time

# 各問題を解き、問題番号と式を表示
for no, input_nums, create_num in QUERY:
    formula = solver(input_nums, create_num)
    print(f"[No.{no}] {formula} = {create_num}")

[No.1] 1 + (2 * (3 + 4)) = 15
[No.2] (4 - (9 / 8)) * 8 = 23
[No.3] (3 * 9) + (5 - 8) = 24
[No.4] ((1 + 7) * 3) - 5 = 19
[No.5] 1 * (2 * (2 * (6 + (7 - 9)))) = 16
[No.6] 1 - (2 * (3 + (3 - (3 + 9)))) = 13
[No.7] 1 * (4 + (5 + (6 * (6 - 6)))) = 9
[No.8] 1 - ((5 - 7) * 6) = 13
[No.9] 1 * (2 / (3 / (6 * 6))) = 24
[No.10] (2 * (4 + 4)) + 3 = 19
[No.11] 2 + (4 + (7 + (9 / 9))) = 14
[No.12] 3 + (4 + (7 / (8 + (8 - 9)))) = 8
[No.13] 1 + (4 / (3 / 6)) = 9
[No.14] 2 + ((9 / 3) + 9) = 14
[No.15] 1 * (3 + (4 + (3 * 5))) = 22
[No.16] 1 * (6 - (4 - 6)) = 8
[No.17] 1 * (1 - (4 - (6 + (6 + 8)))) = 17
[No.18] 4 + (4 / (5 / (7 + 8))) = 16
[No.19] 3 * ((4 / 3) + 7) = 25
[No.20] (2 * (2 - (6 - 8))) + 9 = 17
[No.21] (1 + (9 / 9)) * 8 = 16
[No.22] 2 + (5 + (5 - (6 / (6 - 8)))) = 15
[No.23] 2 + (2 * (3 - (6 - 8))) = 12
[No.24] 6 + (6 - (7 - (8 + 9))) = 22
[No.25] (1 + 8) * (8 - 6) = 18
[No.26] (2 * (5 + (7 - 7))) + 8 = 18
[No.27] (3 * (8 - 3)) + 4 = 19
[No.28] 5 - (9 / (6 - 9)) = 8
[No.29] 1 + ((2 * (3 + (4 

## 実際に求めた式があっているかを確認する(pytest)

GoogleColab上でpytestを実施する際、テスト対象のコードをスクリプト化(ファイル保存)する必要があります。  
以下のコードをファイル保存します。

+ 逆ポーランド記法の演算処理
+ QuizKnockパズル用ソルバー
+ 実際に動画の問題を解いてみる
+ 下記のテストコード


### 検証対象のコードをスクリプト化

In [15]:
#@title calc_RPN関数のスクリプト化(calc_rpn.py)
%%writefile calc_rpn.py
from fractions import Fraction


def calc_RPN(rpn: str, return_formula: bool=False) -> Fraction|float|str:
    """逆ポーランド記法の演算を実施する

    Args:
        rpn (str): 逆ポーランド記法
        return_formula (bool): Trueの場合は計算結果、Falseの場合は四則演算式の文字列を返却 (Default: False)

    Returns:
        Fraction|float|str: 演算結果(ゼロ割があった場合はfloat("inf")) or 四則演算式の文字列
    """
    def ope(a, b, op):
        match op:
            case "+": return a + b
            case "-": return a - b
            case "*": return a * b
            case "/": return a / b if b != 0 else float("INF")
            case _:
                raise ValueError(f"op(char) is invalid value (value: {op})")

    ops = "+-*/"
    stack = []
    for i, c in enumerate(rpn):
        # 数値(演算子ではない)の場合、スタック
        if c not in ops:
            stack.append(Fraction(int(c), 1))
        # 演算子の場合は、スタックから数値を2つ取り出して演算する
        else:
            # 演算子に対して、数値が不足している場合は例外を出す
            if len(stack) < 2:
                raise ValueError(f"There is an error in reverse polish notation (notation: {rpn}, index: {i}, char: {c})")

            var1 = stack.pop()
            var2 = stack.pop()

            # 式文字列を返却するときの処理
            if return_formula:
                result = f"{var2} {c} {var1}"
                if i < len(rpn) - 1:
                    result = f"({result})"
                stack.append(result)
            # 計算結果を返却するときの処理
            else:
                result = ope(var2, var1, c)
                # 演算結果が無限(ゼロ割)の場合、無限を返却
                if result == float("inf"):
                    return float("inf")
                stack.append(result)

    # 数値に対して、演算子が不足している場合は例外を出す
    if len(stack) > 1:
        raise ValueError("Insufficient number of operators in reverse polish notation")

    # 計算結果 もしくは 数式を返却
    return stack[0] if return_formula else str(stack[0])

Overwriting calc_rpn.py


In [16]:
#@title solver関数のスクリプト化(quizknock_solver.py)
%%writefile quizknock_solver.py
from calc_rpn import calc_RPN

def solver(input_nums: list[int | str], create_num: int) -> str|None:
    """
    複数の数値と四則演算を用いて、指定した数値を作成するソルバー

    Args:
        input_nums (list[int | str]): 入力数値リスト(数値は2つ以上)
        create_num (int): 作成する数値

    Returns:
        str|None: 指定した数値を作成できる四則演算式の文字列 (解がない場合はNone)
    """
    n_nums = len(input_nums)    # 入力数値の個数
    n_ops = n_nums - 1          # 演算子の個数
    n_rpn = n_nums + n_ops      # 逆ポーランド記法の文字数

    used = [False] * n_nums     # 使用したinput_numsを記憶する変数

    # DFSにより、あり得る逆ポーランド記法を探索し、create_numと一致する式の場合は、その式(文字列)を返却する
    # rpn: 作成中の逆ポーランド記法
    # n_stack: スタックしている数値の個数
    def dfs(rpn:str = "", n_stack: int = 0) -> str|None:
        nonlocal used

        # 生成した逆ポーランド記法の計算を実施し、create_numと一致した式(文字列)を返却
        if len(rpn) >= n_rpn:
            if calc_RPN(rpn) == str(create_num):
                return calc_RPN(rpn, return_formula=True)
            else:
                return None

        appeared = set()        # 重複防止のために、出現した数値を保存する変数

        for i, num in enumerate(input_nums):
            # 未使用 かつ 未出現の数値の場合、追加可能
            if (not used[i]) and (num not in appeared):
                appeared.add(num)
                used[i] = True
                formula = dfs(rpn + str(num), n_stack + 1)
                # 該当する式が見つかった場合は式を返却
                if formula:
                    return formula
                used[i] = False

        # スタックが2つ以上ある場合は演算子が追加可能
        if n_stack >= 2:
            for op in "+-*/":
                formula = dfs(rpn + op, n_stack - 1)
                # 該当する式が見つかった場合は式を返却
                if formula:
                    return formula

        # create_numと一致する式が見つからない場合はNoneを返却
        return None

    return dfs()

Overwriting quizknock_solver.py


In [17]:
#@title 問題データのスクリプト化(sample_input.py)
%%writefile sample_input.py
# 問題 (問題番号、入力となる数値、作成する数値)
QUERY = [
    (1, [1, 2, 3, 4], 15),        (2, [4, 8, 8, 9], 23),        (3, [3, 5, 8, 9], 24),     (4, [1, 3, 5, 7], 19),        (5, [1, 2, 2, 6, 7, 9], 16),
    (6, [1, 2, 3, 3, 3, 9], 13),  (7, [1, 4, 5, 6, 6, 6], 9),   (8, [1, 5, 6, 7], 13),     (9, [1, 2, 3, 6, 6], 24),     (10, [2, 3, 4, 4], 19),
    (11, [2, 4, 7, 9, 9], 14),    (12, [3, 4, 7, 8, 8, 9], 8),  (13, [1, 3, 4, 6], 9),     (14, [2, 3, 9, 9], 14),       (15, [1, 3, 4, 3, 5], 22),
    (16, [1, 4, 6, 6], 8),        (17, [1, 1, 4, 6, 6, 8], 17), (18, [4, 4, 5, 7, 8], 16), (19, [3, 3, 4, 7], 25),       (20, [2, 2, 6, 8, 9], 17),
    (21, [1, 8, 9, 9], 16),       (22, [2, 5, 5, 6, 6, 8], 15), (23, [2, 2, 3, 6, 8], 12), (24, [6, 6, 7, 8, 9], 22),    (25, [1, 6, 8, 8], 18),
    (26, [2, 5, 7, 7, 8], 18),    (27, [3, 3, 4, 8], 19),       (28, [5, 6, 9, 9], 8),     (29, [1, 2, 3, 4, 8, 9], 22), (30, [1, 2, 7, 8], 17),
    (31, [1, 1, 3, 6, 9], 14),    (32, [2, 6, 7, 9], 21),       (33, [1, 4, 6, 7], 23),    (34, [1, 4, 4, 6, 8], 12),    (35, [1, 3, 5, 9], 14),
    (36, [1, 1, 3, 5, 5, 7], 24), (37, [2, 2, 5, 6], 12),       (38, [1, 6, 8, 8], 18),    (39, [1, 3, 4, 6], 13),       (40, [2, 6, 6, 8, 8], 19),
    (41, [3, 4, 4, 9], 11),       (42, [1, 4, 6, 9], 17),       (43, [4, 7, 8, 8], 10),    (44, [1, 6, 8, 9], 18),       (45, [3, 4, 6, 8], 18),
    (46, [1, 2, 4, 8, 9], 13),    (47, [3, 7, 8, 8], 18),       (48, [1, 2, 3, 3, 6], 18), (49, [4, 5, 9, 9, 9], 17),    (50, [5, 6, 7, 9], 19),
    (51, [3, 6, 6, 8], 21),       (52, [1, 2, 7, 7, 8], 17),    (53, [2, 3, 6, 6], 19)
]

Overwriting sample_input.py


### 検証コードを実行

In [18]:
%%writefile test_solver.py
import pytest

# ファイル保存した関数およびデータの準備
from sample_input import QUERY
from quizknock_solver import solver


# solverで求めた式から値を計算し、指定した値が作成できてるかをテスト
@pytest.mark.parametrize("no, input_nums, create_num", QUERY)
def test_solver(no, input_nums, create_num):
    formula = solver(input_nums, create_num)
    assert formula != None

    result = eval(formula)
    assert result == create_num

Overwriting test_solver.py


In [19]:
# テストを実行
!pytest test_solver.py

======================================= test session starts ========================================
platform linux -- Python 3.10.12, pytest-7.4.4, pluggy-1.4.0
rootdir: /content
plugins: anyio-3.7.1
collected 53 items                                                                                 

test_solver.py .....................................................                         [100%]

======================================== 53 passed in 1.02s ========================================
